**Table of contents**<a id='toc0_'></a>    
- [Install the required libraries](#toc1_)    
- [Import libraries](#toc2_)    
- [Connect to Firebase using Service Account](#toc3_)    
- [Queries](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Install the required libraries](#toc0_)

```bash
!pip install --upgrade firebase-admin
```

# <a id='toc2_'></a>[Import libraries](#toc0_)

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

# <a id='toc3_'></a>[Connect to Firebase using Service Account](#toc0_)

In [2]:
# Use a service account.
cred = credentials.Certificate("aiuda-ffc77-1f6910e96817.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()

# <a id='toc4_'></a>[Queries](#toc0_)

In [3]:
categories = db.collection("categories")
docs = categories.stream()

for doc in docs:
    print(doc.to_dict())

{'active': True, 'rating': 3, 'tags': ['Entregas', 'Medicinas', 'Encargos'], 'imageURL': 'https://firebasestorage.googleapis.com/v0/b/aiuda-ffc77.appspot.com/o/categories%2Fcasual-life-3d-side-view-of-delivery-man-standing-with-bicycle-and-looking-at-the-phone.png?alt=media&token=6dd3392e-5fd1-4b2d-a93e-23a6dc1700b3', 'name': 'Delivery'}
{'rating': 4.5, 'active': True, 'tags': ['Limpieza', 'Pintor', 'Albañil', 'Plomero', 'Electricista'], 'imageURL': 'https://firebasestorage.googleapis.com/v0/b/aiuda-ffc77.appspot.com/o/categories%2Fhandyman.jpeg?alt=media&token=927eee83-9e54-450f-9b51-70155028f02b', 'name': 'Hogar'}
{'rating': 5, 'active': True, 'tags': ['Autos', 'Motos', 'Electronica', 'Bicicletas', 'Aires Acondicionados'], 'imageURL': 'https://firebasestorage.googleapis.com/v0/b/aiuda-ffc77.appspot.com/o/categories%2F1024.png?alt=media&token=922cd380-5a90-498e-a262-58f57648229d', 'name': 'Mecanico'}
{'active': True, 'rating': 4, 'tags': ['Computadoras'], 'imageURL': 'https://www.muyc

# LLM in Action

## Create a tool to connect to firebase

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.agents import load_tools, initialize_agent, AgentType

In [6]:
llm = ChatOpenAI(openai_api_key=api_key)

In [7]:
tools = []

desc = (
    "use this tool when a user ask for a categories "
    "It will return the user profile in JSON format."
)


class ServiceProvider(BaseTool):
    name = "Categories"
    description = desc

    def _run(self, service: str):
        users_ref = (
            db.collection("categories")
            .where(filter=FieldFilter("active", "==", True))
            .where(filter=FieldFilter("rating", ">=", 3))
        )
        docs = users_ref.stream()

        providers = []
        for doc in docs:
            print(f'{doc.id} => {doc.to_dict()["name"]}')
            providers.append(doc.to_dict()["name"])
        return providers

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


tools.append(ServiceProvider())

In [8]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

In [9]:
# initialize agent with tools
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=conversational_memory,
)

/home/nmlemus/miniforge3/envs/llm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent.invoke(
    "Puedes buscar las categorias disponibles? \
    Por favor, devuelveme la respuesta en español"
)



> Entering new AgentExecutor chain...
```json
{
    "action": "Categories",
    "action_input": "Puedes buscar las categorias disponibles?"
}
```delivery => Delivery
transporte => Transporte
reparaciones => Reparaciones
spa => SPA
hogar => Hogar
mecanico => Mecanico
salud => Salud

Observation: ['Delivery', 'Transporte', 'Reparaciones', 'SPA', 'Hogar', 'Mecanico', 'Salud']
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Las categorías disponibles son: 'Delivery', 'Transporte', 'Reparaciones', 'SPA', 'Hogar', 'Mecanico', 'Salud'."
}
```

> Finished chain.


{'input': 'Puedes buscar las categorias disponibles?     Por favor, devuelveme la respuesta en español',
 'chat_history': [],
 'output': "Las categorías disponibles son: 'Delivery', 'Transporte', 'Reparaciones', 'SPA', 'Hogar', 'Mecanico', 'Salud'."}

In [11]:
agent.invoke(
    "Ahora puedes decirme sin usar ninguna herramienta si existe alguna catetoria que pueda usar para pedir algo a domicilio?"
)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "La categoría 'Delivery' es la que puedes usar para pedir algo a domicilio."
}
```

> Finished chain.


{'input': 'Ahora puedes decirme sin usar ninguna herramienta si existe alguna catetoria que pueda usar para pedir algo a domicilio?',
 'chat_history': [HumanMessage(content='Puedes buscar las categorias disponibles?     Por favor, devuelveme la respuesta en español'),
  AIMessage(content="Las categorías disponibles son: 'Delivery', 'Transporte', 'Reparaciones', 'SPA', 'Hogar', 'Mecanico', 'Salud'.")],
 'output': "La categoría 'Delivery' es la que puedes usar para pedir algo a domicilio."}

In [12]:
agent.invoke("Y para resolver un problema con el auto?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "La categoría 'Mecánico' es la que puedes usar para resolver un problema con el auto."
}
```

> Finished chain.


{'input': 'Y para resolver un problema con el auto?',
 'chat_history': [HumanMessage(content='Puedes buscar las categorias disponibles?     Por favor, devuelveme la respuesta en español'),
  AIMessage(content="Las categorías disponibles son: 'Delivery', 'Transporte', 'Reparaciones', 'SPA', 'Hogar', 'Mecanico', 'Salud'."),
  HumanMessage(content='Ahora puedes decirme sin usar ninguna herramienta si existe alguna catetoria que pueda usar para pedir algo a domicilio?'),
  AIMessage(content="La categoría 'Delivery' es la que puedes usar para pedir algo a domicilio.")],
 'output': "La categoría 'Mecánico' es la que puedes usar para resolver un problema con el auto."}